In [3]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

filepath = tf.keras.utils.get_file('shakespeare.txt',
                             'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

text = text[300_000: 800_000]

characters = sorted(set(text))
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i+SEQ_LENGTH])
    next_char.append(text[i+SEQ_LENGTH])

x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_char[i]]] = 1

model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

model.fit(x, y, batch_size=256, epochs=4)

model.save('textgenerator.model')

C:\Users\hnikolov\AppData\Local\Temp\ipykernel_9624\3232613534.py:29: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool)
C:\Users\hnikolov\AppData\Local\Temp\ipykernel_9624\3232613534.py:30: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(character

Epoch 1/4
651/651 [==============================] - 39s 57ms/step - loss: 2.6987
Epoch 2/4
651/651 [==============================] - 40s 62ms/step - loss: 2.3021
Epoch 3/4
651/651 [==============================] - 41s 62ms/step - loss: 2.1907
Epoch 4/4
651/651 [==============================] - 41s 63ms/step - loss: 2.1104
INFO:tensorflow:Assets written to: textgenerator.model\assets


INFO:tensorflow:Assets written to: textgenerator.model\assets


In [11]:
model = tf.keras.models.load_model('textgenerator.model')

def sample(preds, temperature=1.0):
    preds = np.array(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(lenght, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index+SEQ_LENGTH]
    generated += sentence
    for i in range(lenght):
        x = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1

        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [12]:
print('---0.2---')
print(generate_text(300, 0.2))
print('---0.4---')
print(generate_text(300, 0.4))
print('---0.6---')
print(generate_text(300, 0.6))
print('---0.8---')
print(generate_text(300, 0.8))
print('---0.9---')
print(generate_text(300, 0.9))
print('---1---')
print(generate_text(300, 1))

---0.2---
lling out a suit;
and sometime comes shere the hath the seath the cand the hand the cand the hing the hing the hard the will the pare the will the here the will the his the hard the callore the hand the the pare the sand the cander the hear the hard in the have the will the here the will the hare the ward the his mand and the his the ward
---0.4---
h sight to be shown,
but to rejoice in seare thou for male this deall sis thit with that the wall the hath me he me that the the cand and his the hand it heprist in the cand in the have of the here the and the hald that the prist the the this dowe the beath the will we the ines dowe the hish mace mane thear the king with not the ay the hi
---0.6---
n that side yours.
now is this golden crave the willl thay sterf they mise this the warn, my pich thou hald then sins, the is if and the thoul youllles and whit this not me wind this that the ance whe ind bether will hers ptothen,
thou hear he fay the eave to nors with sould hee what the